In [1]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_core.tools.structured import StructuredTool
from langchain_core.documents import Document
from dotenv import load_dotenv
from langchain.chains.llm import LLMChain
from rich import print

load_dotenv()

True

In [2]:
import random
from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """
    Retrieve the current weather for a specified location.

    Args:
        location (str): The name of the location for which to retrieve weather information.

    Returns:
        str: A brief description of the current weather conditions at the specified location.
    """
    if random.choice([True,False]):
        return "The weather is cool and foggy with a mild temperature."
    else:
        return "The weather is warm and sunny with clear skies."


@tool
def get_coolest_cities() -> str:
    """
    Retrieve a list of cities that are considered cool or popular.

    Returns:
        str: A comma-separated list of popular cities known for their unique appeal.
    """
    return "New York City, San Francisco, Tokyo, Paris, Berlin"


@tool
def get_mom_status(query: str) -> str:
    """
    Respond to inquiries about mom's status or availability.

    Args:
        query (str): A specific question or context related to mom.

    Returns:
        str: A status update or response based on the inquiry.
    """
    return random.choice(["Available", "Unavailable", "Busy", "Free", "Not reachable"])


@tool
def get_dad_status(query: str) -> str:
    """
    Respond to inquiries about dad's status or availability.

    Args:
        query (str): A specific question or context related to dad.

    Returns:
        str: A status update or response based on the inquiry.
    """
    return random.choice(["Available", "Unavailable", "Busy", "Free", "Not reachable"])

@tool
def get_sister_status(query: str) -> str:
    """
    Respond to inquiries about sister status or availability.

    Args:
        query (str): A specific question or context related to sister.

    Returns:
        str: A status update or response based on the inquiry.
    """
    return random.choice(["Available", "Unavailable", "Busy", "Free", "Not reachable"])


@tool
def get_github_project_info(query: str) -> str:
    """
    Retrieve information about projects from a GitHub repository.

    Args:
        query (str): A keyword or description of the type of project information requested.

    Returns:
        str: A brief summary of GitHub projects associated with the user or query.
    """
    return "Your GitHub portfolio includes projects such as a proxy checker, a Telegram views booster, and a reaction enhancer."


@tool
def get_local_project_info(query: str) -> str:
    """
    Retrieve information about projects currently being developed locally.

    Args:
        query (str): A keyword or description of the project information being requested.

    Returns:
        str: A summary of active projects or ongoing developments on the local system.
    """
    return "Currently, you are working on building an autonomous agent for web crawling and data extraction."

In [3]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama

from langchain_google_genai import ChatGoogleGenerativeAI


model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
model_embedding = OllamaEmbeddings(model="nomic-embed-text:latest")
vectore_store = Chroma(
    embedding_function=model_embedding, persist_directory="./functionCalling"
)

c:\Users\roshan.yadav\AppData\Local\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tools_func = [
    get_weather,
    get_coolest_cities,
    get_mom_status,
    get_dad_status,
    get_github_project_info,
    get_local_project_info,
    get_sister_status
]
AvailableTools = {tool.name: tool for tool in tools_func}
toolDoc = [
    Document(
        page_content=f"{tool.name} :\n\n{tool.description}",
        metadata={"func_name": tool.name},
    )
    for tool in tools_func
]
ids=vectore_store.add_documents(documents=toolDoc)

In [9]:
result=[doc for doc,score in vectore_store.similarity_search_with_score("Are my parents available this evening?", k=len(tools_func)) if score < 1]
result

[Document(metadata={'func_name': 'get_dad_status'}, page_content="get_dad_status :\n\nRespond to inquiries about dad's status or availability.\n\nArgs:\n    query (str): A specific question or context related to dad.\n\nReturns:\n    str: A status update or response based on the inquiry."),
 Document(metadata={'func_name': 'get_mom_status'}, page_content="get_mom_status :\n\nRespond to inquiries about mom's status or availability.\n\nArgs:\n    query (str): A specific question or context related to mom.\n\nReturns:\n    str: A status update or response based on the inquiry.")]

In [ ]:
[score for _,score in result]

In [ ]:
vectore_store.delete(ids)

In [ ]:
raise ValueError("ok")


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompt_values import ChatPromptValue

prompt = ChatPromptTemplate(
    [
        SystemMessage(
            content="You are a personal assistant for Roshan Yadav, aware of all his professional and personal details, and your role is to assist with tasks, offer advice, and provide solutions based on his goals, preferences, and ongoing projects."
        ),
        HumanMessage(
            content="Are my parents available this evening? If they are, we can plan dinner. Could you please check and confirm if it's possible?"
        ),
    ]
)


def llm_with_tool(data: ChatPromptValue):
    accessable_tools = []
    humanMsg: HumanMessage = data.messages[-1]
    related_tools = vectore_store.similarity_search(humanMsg.content, k=3)
    for function in related_tools:
        tool_name = function.metadata.get("func_name")
        accessable_tools.append(AvailableTools[tool_name])

    return model.bind_tools(accessable_tools)

In [ ]:
chain = prompt | llm_with_tool
resp = chain.invoke({})
print(resp)

In [7]:
from langchain_core.messages.tool import ToolCall
from google.generativeai.types import Tool
from typing import Dict


def callTools(tool_calls: ToolCall, AvailableTools: Dict[str, StructuredTool]):
    toolCallResult = list()
    for tool_call in tool_calls:
        selected_tool = AvailableTools[tool_call["name"].lower()]
        print(f"Calling -----> {selected_tool.name}")
        tool_msg: ToolMessage = selected_tool.invoke(tool_call)
        toolCallResult.append(tool_msg)
    print(toolCallResult)
    return toolCallResult